<a href="https://colab.research.google.com/github/mobadata/detection-nouveautes-avis/blob/main/lda_detection_nouveautes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import re
from collections import Counter
from types import SimpleNamespace

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer as wnl
from nltk.tokenize import word_tokenize

## Configs

In [ ]:
cfg = SimpleNamespace(**{})

In [ ]:
cfg.data_path = "/content/drive/MyDrive/customers_feedbacks_trends/data/inputs/Electronics_5.json"
cfg.stop_words = set(stopwords.words('english'))
cfg.vect_params = {"ngram_range": (1, 2), "min_df": 10, "max_df": 0.9}
cfg.lda_grid_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

## Import data

In [ ]:
reviews_df = pd.read_json(cfg.data_path, lines=True)

## preprocessing

In [ ]:
def nettoyer_texte(texte, stop_words):
  """low all documents, remove special chars, tokenize and lemmatize them then remove stopwords """
  lemmatizer = wnl()
  if isinstance(texte, str):
    texte = texte.lower()
    texte = re.sub(r"[^a-z\s]", "", texte)
    tokens = word_tokenize(texte)
    lemma = [lemmatizer.lemmatize(word=token) for token in tokens]
    mots_utiles = [mot for mot in lemma if mot not in stop_words]
    return " ".join(mots_utiles)
  else:
        return ""

In [ ]:
reviews_df["review_clean"] = reviews_df["reviewText"].apply(lambda review: nettoyer_texte(review, cfg.stop_words))

In [29]:
reviews_df["review_clean"]

,review_clean
0,got gps husband otr road trucker impressed shi...
1,im professional otr truck driver bought tnd tr...
2,well say ive unit truck four day prior garmin ...
3,going write long review even thought unit dese...
4,ive mine year got try route non truck route te...
...,...
1689183,burned listening hour sine wave app ive learne...
1689184,people like dj style headphone earbud style he...
1689185,im big fan brainwavz actually headphone yet di...
1689186,ive used thebrainwavz ear headphone outstandin...


In [ ]:
count_vectorizer = CountVectorizer(**cfg.vect_params)
text_counts = count_vectorizer.fit_transform(reviews_df["review_clean"])
count_df = pd.DataFrame(text_counts.toarray(), columns = count_vectorizer.get_feature_names())

In [ ]:
lda = LatentDirichletAllocation()